# Hunter Participation by State (WIP)

A look at trends of hunter participation by state and region in the United States based on hunting license information from 2004 to 2018.

## Data Source

Hunting license data are from U.S. Fish and Wildlife Service's [Wildlife and Sport Fish Restoration Program](https://wsfrprograms.fws.gov/Subpages/LicenseInfo/Hunting.htm) page.

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# import sys
# !{sys.executable} -m pip install folium

In [3]:
import folium

In [11]:
# Excel spreadsheet path
excel_path = './data/NatlHuntingLicenseReportAllYearsClean.xlsx'
index_col = 'State'

# Import the number of paid license holders by state
# A paid license holder is one individual regardless of the number of licenses purchased
license_holders = pd.read_excel(excel_path,
                             sheet_name='LicenseHolders',
                             index_col=index_col)
license_holders['2004-18 Change (%)'] = (license_holders[2018] - license_holders[2004]) / license_holders[2004] *100
license_holders['2008-18 Change (%)'] = (license_holders[2018] - license_holders[2008]) / license_holders[2008] *100
license_holders['2013-18 Change (%)'] = (license_holders[2018] - license_holders[2013]) / license_holders[2013] *100

In [12]:
license_holders.head()

,Region,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2004-18 Change (%),2008-18 Change (%),2013-18 Change (%)
State,,,,,,,,,,,,,,,,,,,
AK,Pacific,108921,108487,106916,107131,107260,101547,106653,101750,97858,96979,96979,99954,98084,99121,97537,11.671468,12.314006,7.261662
AL,East South Central,547905,548829,565139,507926,507403,530127,527713,540098,264710,264640,255923,259241,267354,270229,273638,100.229866,114.089785,3.353536
AR,West South Central,326559,340200,328542,326779,382212,390554,385275,390932,381521,380931,354042,375737,378162,386559,402493,-18.865918,-7.762638,-16.385698
AZ,Mountain,305214,324553,215444,200092,195213,195664,194727,191834,201627,202976,202976,196706,182044,181467,183478,66.349099,50.369502,55.988838
CA,Pacific,280967,284069,287147,283539,283539,281472,289609,293263,296623,300615,295163,299303,305962,315588,320092,-12.223048,-4.809546,-0.179414


In [13]:
# Import the number of total licenses by state
# Persons who hunted in more than one state are counted in each state where they hunted
total_licenses = pd.read_excel(excel_path,
                             sheet_name='TotalLicenses',
                             index_col=index_col)
total_licenses['2004-18 Year Change (%)'] = (total_licenses[2018] - total_licenses[2004]) / total_licenses[2004] *100
total_licenses['2008-18 Year Change (%)'] = (total_licenses[2018] - total_licenses[2008]) / total_licenses[2008] *100
total_licenses['2013-18 Year Change (%)'] = (total_licenses[2018] - total_licenses[2013]) / total_licenses[2013] *100

In [14]:
total_licenses.head()

,Region,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2004-18 Year Change (%),2008-18 Year Change (%),2013-18 Year Change (%)
State,,,,,,,,,,,,,,,,,,,
AK,Pacific,352780,402002,362739,310644,287352,278400,353266,250073,239557,227588,227588,227701,223048,223215,222544,58.521461,55.008173,26.716954
AL,East South Central,579944,587686,604043,599876,597095,609156,607121,625962,287455,291183,283899,289660,302220,299224,303217,91.263682,104.278282,-4.795488
AR,West South Central,517046,533585,521098,498919,492479,475057,468656,481943,434479,424003,388698,421548,422822,438108,464563,11.297284,33.019980,8.838729
AZ,Mountain,517747,540038,446402,422136,409672,414623,409680,411617,419161,428669,428669,415432,391973,396299,406210,27.457965,20.780136,24.871751
CA,Pacific,1030160,1054000,1035351,847348,847348,834739,875297,828735,836694,841239,830700,831531,833977,858691,868209,18.653458,24.011075,23.411030


In [15]:
# New England states subset
# Used paid license holders (individual hunters) vs total licenses
#   because that set had missing data in RI for 2013
ne_states = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT']
ne_hunters = license_holders.loc[ne_states]
ne_hunters

,Region,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2004-18 Change (%),2008-18 Change (%),2013-18 Change (%)
State,,,,,,,,,,,,,,,,,,,
CT,New England,37489,39488,42924,42535,41294,44178,46582,50066,49493,48857,48857,52207,54130,54926,57720,-35.050243,-23.267904,-15.141020
ME,New England,163191,166051,168890,165781,175196,189120,191280,193436,196160,195568,195568,199102,205600,201136,197908,-17.541989,-16.555367,-13.710343
MA,New England,57921,57973,59669,56797,61204,57641,57346,59470,57153,59158,57193,59016,69500,56662,68488,-15.428980,1.272883,0.485765
NH,New England,58099,59318,61556,59068,56411,59301,59420,59154,59768,61076,61076,60737,62587,63975,67586,-14.036931,-4.874255,-2.026947
RI,New England,8209,8797,8978,8624,8551,8605,8798,8858,9075,9075,9075,8940,9302,10691,10750,-23.637209,-9.542700,-4.601976
VT,New England,69943,71807,74219,72930,74966,80650,82307,83681,84044,84044,83708,81265,86512,90110,92138,-24.088867,-16.444067,-13.275883


In [16]:
ten_yr_holders = license_holders['2008-18 Change (%)']  #.sort_values(by='2008-18 Change (%)')
print(ten_yr_holders.min())
print(ten_yr_holders.max())

-23.399582738552763
114.08978481808983


In [10]:
# Create chloropleth map

us_states = './Data/us-states.json'
m = folium.Map(location=[43, -102], zoom_start=4)

threshold_scale = [-24, -15, -5, 0, 10, 30, 115]
m.choropleth(geo_data=us_states, data=ten_yr_holders,
#              columns=['2008-18 Change (%)'],
             name='choropleth',
             key_on='feature.id',
             fill_color='RdYlGn',
             fill_opacity=0.7,
             line_opacity=0.2,
             legend_name='Hunting License Holders 10-Year Change',
             threshold_scale=threshold_scale)
m

/Applications/anaconda/envs/py3ds/lib/python3.7/site-packages/folium/folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


## Hunting License Cost Differential between Residents and Non-Residents

Typically, residents enjoy better hunting access in their respective states compared non-residents in two ways. Resident pay less for hunting licenses, tags, permits, and stamps versus what non-residents pay, and they tend to claim larger percentages for any tags that aren't available over-the-counter (one that must be drawn).

The general argument for why residents deserve more and cheaper access is that wildlife management is the state's responsibility - both to execute and (probably more importantly) pay for at the state level. There are several major sources of cash that fund these departments (see the [../ConservationFunding/ConservationFundingSources.ipynb](Conservation Funding project) for a general overview), one of which is the money collected from hunting licenses, tags, permits, and stamps. However, it appears that for FY2018, 14 states actually collected a total dollar amount more from non-residents than they did from residents.

In [20]:
# Get 2018 Data
data_2018 = pd.read_excel(excel_path,
                          sheet_name='2018-Full',
                          index_col=index_col)
neg_states = len(data_2018[data_2018['Non-resident Cost Factor'] < 0])
print('Number of states collecting more from non-residents: {}'.format(neg_states))
data_2018

Number of states collecting more from non-residents: 14


,2018 Paid Hunting License Holders,"2018 Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Non-Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Total Hunting Licenses, Tags, Permits, and Stamps","2018 Cost - Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Cost - Non-Resident Hunting Licenses, Tags, Permits, and Stamps",2018 Gross Cost - Hunting Licenses,Non-resident Cost Factor
State,,,,,,,,
AK,108921,278248,74532,352780,2922692,5568901,8491593,-1.905401
AL,547905,542838,37106,579944,5693734,6107717,11801451,-1.072709
AR,326559,380102,136944,517046,8186097,10579420,18765517,-1.292364
AZ,305214,456140,61607,517747,13358303,4515780,17874083,2.958139
CA,280967,1007422,22738,1030160,20609712,1006282,21615994,20.481050
CO,294319,459518,109246,568764,12751122,44305473,57056595,-3.474633
CT,37489,124169,4063,128232,2085367,394950,2480317,5.280078
DE,17847,52315,7125,59440,632742,372462,1005204,1.698810
FL,190232,313554,18155,331709,7890888,984031,8874919,8.018942
